In [1]:
from getpass import getpass

# We will need pandas for DataFrames and getpass for login
import pandas as pd

# Import our Python client
from hawc_client import HawcClient

### Login

In [2]:
# Local login
domain = "http://127.0.0.1:8000"
username = getpass("Username: ")
password = getpass("Password: ")
client = HawcClient(domain)
client.authenticate(username, password)
print("Authentication successful")

Authentication successful


### Map source study to destination study

In [3]:
# Get HERO IDs from excel
excel_df = pd.read_excel("data/PFAS Epi studies to copy over from other assessments.xlsx")
hero_ids = excel_df["HERO ID"].to_list()

In [4]:
# Set source and destination assessment ids
src_assessment_id = 100500071
dst_assessment_id = 100000026

In [5]:
# Get studies for src and dst (objects are lists of dicts)
src_studies = client.session.get(
    f"{domain}/study/api/study/?assessment_id={src_assessment_id}"
).json()
src_study_ids = [study["id"] for study in src_studies]
dst_studies = client.session.get(
    f"{domain}/study/api/study/?assessment_id={dst_assessment_id}"
).json()
dst_study_ids = [study["id"] for study in dst_studies]

# Get references for src and dst (objects are dataframes)
src_refs = client.lit.reference_ids(src_assessment_id)
dst_refs = client.lit.reference_ids(dst_assessment_id)

# Get matching studies for src and dst (objects are dataframes)
src_hero_studies = src_refs.loc[
    (src_refs["reference_id"].isin(src_study_ids)) & (src_refs["hero_id"].isin(hero_ids))
]
dst_hero_studies = dst_refs.loc[
    (dst_refs["reference_id"].isin(dst_study_ids)) & (dst_refs["hero_id"].isin(hero_ids))
]

In [6]:
# Any missing studies?
src_missing_hero = set(hero_ids) - set(src_hero_studies["hero_id"].to_list())
dst_missing_hero = set(hero_ids) - set(dst_hero_studies["hero_id"].to_list())
print(
    f"Missing source studies for the following HERO IDs: {', '.join([str(id) for id in src_missing_hero])}"
)
print(
    f"Missing destination studies for the following HERO IDs: {', '.join([str(id) for id in dst_missing_hero])}"
)

Missing source studies for the following HERO IDs: 
Missing destination studies for the following HERO IDs: 


**Please make sure there are no missing studies before continuing**

In [7]:
# Create the study mapping
src_hero_studies = src_hero_studies.rename(columns={"reference_id": "src_id"})[
    ["src_id", "hero_id"]
]
dst_hero_studies = dst_hero_studies.rename(columns={"reference_id": "dst_id"})[
    ["dst_id", "hero_id"]
]
hero_studies = src_hero_studies.merge(dst_hero_studies, on="hero_id")
src_to_dst_study = {
    int(src_id): int(dst_id)
    for src_id, dst_id in hero_studies[["src_id", "dst_id"]].to_records(index=False)
}

### Map source to destination metrics

In [8]:
src_metrics = client.session.get(
    f"{domain}/rob/api/metric/?assessment_id={src_assessment_id}"
).json()
dst_metrics = client.session.get(
    f"{domain}/rob/api/metric/?assessment_id={dst_assessment_id}"
).json()

In [11]:
# Try to map src to dst metric by name
src_metric_name_to_id = {metric["name"]: metric["id"] for metric in src_metrics}
dst_metric_name_to_id = {metric["name"]: metric["id"] for metric in dst_metrics}
src_to_dst_metric = {
    src_metric_name_to_id[name]: dst_metric_name_to_id[name]
    for name in src_metric_name_to_id.keys()
}

### Run the risk of bias copy

In [14]:
client.riskofbias.bulk_rob_copy(
    src_assessment_id=src_assessment_id,
    dst_assessment_id=dst_assessment_id,
    dst_author_id=1,
    src_dst_study_ids=list(src_to_dst_study.items()),
    src_dst_metric_ids=list(src_to_dst_metric.items()),
    copy_mode=2,
    author_mode=2,
)

{'log_id': 3,
 'log_url': '/assessment/api/logs/3/',
 'mapping': {'study': {'100517572': 100000936,
   '100517574': 100000939,
   '100517575': 100000940,
   '100517577': 100000943,
   '100517579': 100000946,
   '100517583': 100000950,
   '100517584': 100000951,
   '100517586': 100000954,
   '100517591': 100000960,
   '100517594': 100000977,
   '100517595': 100000978,
   '100517596': 100000979,
   '100517597': 100000980,
   '100517598': 100000981,
   '100517600': 100000983,
   '100517604': 100000988,
   '100517608': 100000994,
   '100517609': 100000995,
   '100517614': 100001000,
   '100517615': 100001001,
   '100517616': 100001002,
   '100517617': 100001003,
   '100517620': 100001023,
   '100517624': 100001029,
   '100517633': 100001039,
   '100517634': 100001040,
   '100517635': 100001041,
   '100517637': 100001044,
   '100517638': 100001045,
   '100517639': 100001046,
   '100517644': 100001051,
   '100517651': 100001059,
   '100517660': 100001069,
   '100517679': 100001325,
   '10051